https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html#scipy.integrate.odeint

In [141]:
import numpy as np
import k3d
from ratelimiter import RateLimiter
from scipy.integrate import odeint

def pend(y, t, b, c):
    theta, omega = y
    dydt = [omega, -b*omega - c*np.sin(theta)]
    return dydt

b = 0.25
c = 5.0
y0 = [np.pi - 0.1, 0.0]
t = np.linspace(0, 30, 600)
sol = np.float32(odeint(pend, y0, t, args=(b, c)))

plot = k3d.plot(grid_auto_fit=False, camera_auto_fit=False, grid=(-4,-2,-2,0,2,1))
point = k3d.points([0, np.sin(sol[0,0]), -np.cos(sol[0,0])], point_size=0.1)
line = k3d.line([0, np.sin(sol[0,0]), 0])
texture = k3d.texture(open('Grid_Pattern.jpg', 'br').read(), 'jpg', translation=[-1.5,0,-1.02])
texture.model_matrix = texture.model_matrix * np.array([[3,3,1,1]], dtype=np.float32)
plot += point + line +  texture
plot.display()

Output()

In [142]:
plot.camera = [-1.6592067371682246,-1.3324933300147228,3.8707220547375956,-2,0,-0.5,-0.42139176249748234,0.8381023080549331,0.3464296519241023]

rate_limiter = RateLimiter(max_calls=1, period=1/30)

for i in range(1, sol.shape[0]):
    with rate_limiter:
        point.positions = [0, np.sin(sol[i,0]), -np.cos(sol[i,0])]
        translation = np.array([0.01,0,0])*np.array([np.arange(i)[::-1]]).T
        line_values = np.float32(np.array([np.zeros(i), np.sin(sol[:i,0]), -np.ones(i)]).T - translation)
        if i-300 < 0: bias = 0
        else: bias = i-300
        line.vertices = line_values[bias:]